## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [6]:
import numpy as np

N = 1000000
A = np.random.randint(0, 1001, size=N)
B = A + 100
mean_B = np.mean(B)

print(f"Среднее значение массива B: {mean_B:.2f}")
print(A)
print(B)

Среднее значение массива B: 600.53
[242 814 556 ... 422 232 766]
[342 914 656 ... 522 332 866]


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [22]:
import random
from random import randint
import string
key = [''.join(random.choices(string.ascii_letters, k=5)).lower() for _ in range(2000000)]
dt = pd.DataFrame(np.random.randint(0,1001, size=(2000000,4)))
dt['key'] = key
print(dt)
if dt.loc[dt['key'] == 'abcde'].shape == (0,0):
    print(dt.loc[dt['key']=='abcde'])
    print('Нет строк с ключем, содержащим первые 5 букв английского алфавита')
else:
    print(dt.loc[dt['key']=='abcde'])

           0    1    2    3    key
0        639  647  759  809  jibvq
1        438  309  977  369  mrrkb
2         52  991  400  830  yqwxl
3        104  989  948  346  fhvry
4        650  746    7  337  rryes
...      ...  ...  ...  ...    ...
1999995  303  903  604  972  yczcg
1999996  499  471    4  368  zdcbh
1999997   43  484  464  449  dnjpf
1999998  640  969  568  933  unvut
1999999  774  716  207  806  bpsjb

[2000000 rows x 5 columns]
          0    1    2    3    key
148575  839  616  465  606  abcde


## Лабораторная работа 3

In [9]:
!pip install line_profiler


Defaulting to user installation because normal site-packages is not writeable


In [12]:
%load_ext line_profiler
import pandas as pd
import timeit

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [13]:
recipes = pd.read_csv("recipes_sample.csv", parse_dates=["submitted"])
reviews = pd.read_csv("reviews_sample.csv", index_col=0, parse_dates=["date"])
recipes.head()


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [14]:
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [15]:
%%time
#A. 
def kol_rating():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1
kol_rating()

Wall time: 4.22 s


4.4544402182900615

In [16]:
%%time
def sr_znac_B(): #B
    s = 0
    k = 0
    dfB = reviews[reviews['date'].dt.year == 2010]
    for _, row in dfB.iterrows():
        s += row['rating']
    k = dfB.shape[0]
    return s/k
print('Среднее значение столбца rating из таблицы reviews',sr_znac_B())

Среднее значение столбца rating из таблицы reviews 4.4544402182900615
Wall time: 431 ms


In [17]:
%%time
def sr_znac_C(): # C
    return reviews[reviews['date'].dt.year == 2010].rating.mean()
print('Среднее значение столбца rating из таблицы reviews',sr_znac_C())

Среднее значение столбца rating из таблицы reviews 4.4544402182900615
Wall time: 9.98 ms


In [18]:
kol_rating()==sr_znac_B()==sr_znac_C()

True

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [1]:
!pip install line_profiler

Defaulting to user installation because normal site-packages is not writeable


In [24]:

%lprun -f kol_rating kol_rating() #для буквы А

In [26]:
%lprun -f sr_znac_B sr_znac_B() #для буквы B

In [27]:
%lprun -f sr_znac_C sr_znac_C() #для буквы C

In [31]:
%%time
def sr_znac_1B(): #Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год;
    s = 0
    k = 0
    dfB = reviews[reviews['date'].dt.year == 2010]
    return sum(dfB.rating)/dfB.shape[0]
print('Среднее значение столбца rating из таблицы reviews',sr_znac_1B())

Среднее значение столбца rating из таблицы reviews 4.4544402182900615
Wall time: 12 ms


In [32]:
%lprun -f sr_znac_1B sr_znac_1B() #для буквы Б, реализованной без цикла

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [36]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count


In [37]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [38]:
reviews = reviews[reviews.rating !=0]
reviews['mean'] = reviews.groupby('recipe_id').rating.transform('mean') #transform вычисление среднего рейтинга для каждой группы
reviews.head()

,user_id,recipe_id,date,rating,review,mean
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,4.818182
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,5.000000
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",4.000000
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,4.555556
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,5.000000


In [53]:
!pip install numba

Defaulting to user installation because normal site-packages is not writeable


In [55]:
import numba
import numpy as np
%%time
def mA(dt):
    s = 0
    sum_rows = 0
    for index, row in dt.iterrows():
        s += abs(row["rating"] - row['mean']) / row['rating']
        sum_rows += 1
    return s/sum_rows
mA(reviews)
#Numba needs NumPy 1.21 or less

ImportError: Numba needs NumPy 1.21 or less

In [ ]:
%%time
rating = reviews['rating'].to_list()
mean = reviews['mean'].to_list() 
@numba.njit(nopython=True)
def mB(rating, mean): 
    s = 0
    sum_rows = 0
    for i in range(len(rating)):
        s += abs(rating[i] - mean[i]) / rating[i]
        sum_rows += 1
    return s/sum_rows
mB(rating, mean)
#Numba needs NumPy 1.21 or less

In [ ]:
%%time
def mC(dt):
    rating = dt['rating'].to_numpy()
    mean = dt['mean'].to_numpy() 
    mape = abs(rating - mean) / rating
    return mape.mean()
mC(reviews)
#Numba needs NumPy 1.21 or less

In [ ]:
%%time
rating = reviews['rating'].to_numpy()
mean = reviews['mean'].to_numpy() 

@numba.jit
def mD(rating, mean):
    mape = abs(rating - mean) / rating
    return mape.mean()
mD(rating, mean)
#Numba needs NumPy 1.21 or less

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4